# Putting it all together

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Rectangle, Circle, Ellipse, Polygon

from astropy.table import QTable
from astropy import units as u

from astroplan import Observer, FixedTarget
from astroplan.plots import plot_finder_image

from astroquery.gaia import Gaia

import warnings
warnings.filterwarnings("ignore", category = UserWarning)

### Use `FixedTarget` from `astroplan` to get coordinates for a target object

In [ ]:
my_object = "NGC 2682"  # NGC 2682 (M 67) is an open cluster in Cancer.

In [ ]:
my_target = FixedTarget.from_name(my_object)

In [ ]:
my_target.coord

In [ ]:
my_target.dec.degree

In [ ]:
my_target.ra.degree

#### Use `plot_finder_image` to get get an images of the object from the [ESO Online Digitized Sky Survey](http://archive.eso.org/dss/dss).

In [ ]:
ax, hdu = plot_finder_image(my_target, fov_radius= 0.4 * u.deg)

## Get the the [Gaia data](https://www.cosmos.esa.int/web/gaia/dr3) for the object

- Note the use of `f-string` and variables in the query

In [ ]:
my_query = f"""
SELECT TOP 3000
source_id, ra, dec, phot_g_mean_mag, bp_rp, parallax
FROM gaiadr3.gaia_source_lite
WHERE CONTAINS(
   POINT('ICRS', {my_target.ra.degree}, {my_target.dec.degree}),
   CIRCLE('ICRS', ra, dec, 0.3)
   ) = 1
AND parallax > 0.1
AND bp_rp IS NOT NULL
ORDER BY parallax DESC
"""

In [ ]:
print(my_query)

In [ ]:
my_job_query = Gaia.launch_job(my_query)

In [ ]:
print(my_job_query)

In [ ]:
my_table = my_job_query.get_results()

In [ ]:
my_table[0:2]

### Use `parallax` and `phot_g_mean_mag` to get **distance** and **absolute magnitude**

- Add then as columns to the data table

In [ ]:
distance_to_object = my_table['parallax'].to(u.parsec, equivalencies=u.parallax())

my_table['distance'] = distance_to_object

In [ ]:
my_table[0:2]

In [ ]:
def find_absmag(my_gmag, my_distance):
    result = my_gmag - 5 * np.log10( my_distance / (10 * u.parsec)) * u.mag
    return result

In [ ]:
my_table['abs_g'] = find_absmag(my_table['phot_g_mean_mag'], my_table['distance']) * u.mag

In [ ]:
my_table[0:2]

## Plot a histogram of the distances

- All the stars in the cluster should have the same distance
- Objects not in the cluster will have different distances

In [ ]:
fig, ax = plt.subplots(
    figsize = (8, 5), 
    constrained_layout = True
)

ax.set_xlabel("Distance (pc)")
ax.set_ylabel("Number")

ax.hist(my_table['distance'],
        bins = 100,
        histtype = 'stepfilled',
        facecolor = 'MediumOrchid');

## Pretty easy to see the cluster's distance

- Use the **median** of the data as an easy (rough) estimate to the cluster distance.
- Make a subset of the data of all objects within 100 pc of the median
  - `| distance - median | < 100`

In [ ]:
np.median(my_table['distance'])

#### The cluster distance from the literature is 800 - 900 pc, so this value seems fine.

In [ ]:
my_cluster_table = my_table[np.abs(my_table['distance'] - np.median(my_table['distance']) < 100)]

In [ ]:
my_outlier_table = my_table[np.abs(my_table['distance'] - np.median(my_table['distance']) >= 100)]

In [ ]:
len(my_cluster_table), len(my_outlier_table)

## Make a color magnitude diagram (CMD) of the object

In [ ]:
fig, ax = plt.subplots(
    figsize = (15, 15), 
    constrained_layout = True
)

ax.set_ylim(-2.5,12)

ax.set_aspect(1/4)         # Make 1 unit in X = 4 units in Y
ax.invert_yaxis()

ax.set_xlabel("BP - RP",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_ylabel(r"$G_{M}$",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_title(f"{my_target.name}",
             fontfamily = 'serif',
             fontsize = 30)

### Plot Data ###

ax.plot(my_outlier_table['bp_rp'], my_outlier_table['abs_g'],
        color = "red",
        marker = "+",
        linestyle = "None",
        markersize = 5,
        label = "Outliers")

ax.plot(my_cluster_table['bp_rp'], my_cluster_table['abs_g'],
        color = "#4C0B5F",
        marker = "o",
        linestyle = "None",
        markersize = 5,
        label = "Cluster members")

ax.legend(loc=0, shadow=True);

---

# Annotate your plots with `annotate()`

### There are 4 parameters that you use with `annotate()`.

- `text` : the text label (string).
- `xy = (X, Y)` : The coordinates (X, Y) where you want your arrowhead to point to.
- `xytext = (X, Y)` : The coordinates (X, Y) where you want your `text` to display.
- `arrowprops` : A dictionary of key-value pairs which define various properties for the arrow, such as color, size and arrowhead type.
  - [List of Matplotlib Arrowstyles](https://coderslegacy.com/python/matplotlib-customize-arrowstyles/)
  - [List of Matplotlib Arrowprops](https://coderslegacy.com/python/matplotlib-arrowprops/)
  
---

In [ ]:
fig, ax = plt.subplots(
    figsize = (15, 15), 
    constrained_layout = True
)

ax.set_ylim(-2.5,12)

ax.set_aspect(1/4)         # Make 1 unit in X = 4 units in Y
ax.invert_yaxis()

ax.set_xlabel("BP - RP",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_ylabel(r"$G_{M}$",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_title(f"{my_target.name}",
             fontfamily = 'serif',
             fontsize = 30)

### Text Annotation ###

ax.annotate('Main Sequence',
             fontsize = 25,
             color = 'green',
             xy = (2.0, 7.0),
             xytext = (2.0, 5.0),
             arrowprops = {'color' : 'green',
                           'linewidth' : 4,
                           'arrowstyle' : '->, head_length = 0.8, head_width = 0.5'}
           )


ax.annotate('Main\nSequence\nTurnoff',
             fontsize = 25,
             color = 'blue',
             xy = (0.7, 3.3),
             xytext = (-0.3, 6.0),
             arrowprops = {'color' : 'blue',
                           'linewidth' : 4,
                           'arrowstyle' : '-|>, head_length = 0.8, head_width = 0.3'}
           )

ax.annotate('Red Giant\nBranch',
             fontsize = 25,
             color = 'red',
             xy = (1.3, 1.5),
             xytext = (1.7, 0.0),
             arrowprops = {'color' : 'red',
                           'linewidth' : 4,
                           'arrowstyle' : '-|>, head_length = 0.8, head_width = 0.3',
                           'connectionstyle' : 'angle3'}
           )

### Plot Data ###

ax.plot(my_cluster_table['bp_rp'], my_cluster_table['abs_g'],
        color = "#4C0B5F",
        marker = "o",
        linestyle = "None",
        markersize = 5);

---

# Annotate your plots with Shapes

- `Rectangle(center, width, height, **kwargs)`
- `Circle(center, radius, **kwargs)`
- `Ellipse(center, width, height, **kwargs)`
- `Polygon(array of verticies)`

### [Drawing Shapes with Matplotlib Patches](https://coderslegacy.com/python/drawing-shapes-matplotlib-patches/)
---

In [ ]:
fig, ax = plt.subplots(
    figsize = (15, 15), 
    constrained_layout = True
)

ax.set_ylim(-2.5,12)

ax.set_aspect(1/4)         # Make 1 unit in X = 4 units in Y
ax.invert_yaxis()

ax.set_xlabel("BP - RP",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_ylabel(r"$G_{M}$",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_title(f"{my_target.name}",
             fontfamily = 'serif',
             fontsize = 30)

### Text Annotation ###

ax.annotate('Main Sequence',
             fontsize = 25,
             color = 'green',
             xy = (2.0, 7.0),
             xytext = (2.0, 5.0),
             arrowprops = {'color' : 'green',
                           'linewidth' : 4,
                           'arrowstyle' : '->, head_length = 0.8, head_width = 0.5'}
           )


ax.annotate('Main\nSequence\nTurnoff',
             fontsize = 25,
             color = 'blue',
             xy = (0.7, 3.3),
             xytext = (-0.3, 6.0),
             arrowprops = {'color' : 'blue',
                           'linewidth' : 4,
                           'arrowstyle' : '-|>, head_length = 0.8, head_width = 0.3'}
           )

ax.annotate('Red Giant\nBranch',
             fontsize = 25,
             color = 'red',
             xy = (1.3, 1.5),
             xytext = (1.7, 0.0),
             arrowprops = {'color' : 'red',
                           'linewidth' : 4,
                           'arrowstyle' : '-|>, head_length = 0.8, head_width = 0.3',
                           'connectionstyle' : 'angle3'}
           )

### Shape Annotation ###

my_shape_one = Ellipse((0.35, 2.1),
                       width = 0.75,
                       height = 2.85,
                       angle = 0,
                       color = (0, 0, 0.7, 0.2))

ax.add_patch(my_shape_one)

ax.text(-0.1, 0.4,
       'Blue Stragglers',
        color = (0, 0, 0.7, 0.5),
        fontsize = 24);


my_solar_shape = np.array( ((0.85, 5.2), (1.05, 5.2), (0.86, 4.2), (0.70, 4.2)) )

my_shape_two = Polygon(my_solar_shape,
                       facecolor = 'LemonChiffon',
                       edgecolor = 'red',
                       linewidth = 4)

ax.add_patch(my_shape_two)

ax.annotate('Solar-Type\nStars',
             fontsize = 25,
             color = 'OrangeRed',
             xy = (0.85, 5.4),
             xytext = (0.5, 9.0),
             arrowprops = {'facecolor' : 'LemonChiffon',
                           'edgecolor' : 'red',
                           'linewidth' : 4,
                           'arrowstyle' : 'fancy, head_length = 1.8, head_width = 0.8, tail_width=0.5'}
           )

### Plot Data ###

ax.plot(my_cluster_table['bp_rp'], my_cluster_table['abs_g'],
        color = "#4C0B5F",
        marker = "o",
        linestyle = "None",
        markersize = 5);